# Generative AI Evaluation Metrics in MLflow

MLflow 2.8 introduced new [Generative AI Metrics](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#generative-ai-metrics) that use LLMs to evaluate model output text. There are a few different GenAI metrics to choose from:
- [Answer Correctness](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.answer_correctness), which compares a model's output to a ground truth answer
- [Answer Relevance](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.answer_correctness), which evaluates how appropriate and applicable a response is with respect to the input
- [Answer Similarity](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.answer_similarity), which assesses the semantic similarity of a generated response to a ground truth answer
- [Faithfulness](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.faithfulness), which tests the factual similarity of a model's response to some provided context (e.g. in a RAG system)
- [Relevance](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.relevance), which examines the output with respect to the input and provided context (e.g. in a RAG system) and rates its relevance and significance. Note that this differs from the `Answer Similarity` metric, which does not have a context component.

These all work in fudamentally the same way: pick a model and (optionally) define an example, at which point you can use the new metric in the MLflow.evaluate() system. Let's go through a few examples.

In [1]:
# setup
import openai
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Answer Correctness

In [2]:
from mlflow.metrics.genai import EvaluationExample, answer_correctness

example1 = EvaluationExample(
    input="What is MLflow Tracking?",
    output="MLflow Tracking's API and UI log ML workflow aspects like parameters, "
    "code, metrics, and artifacts. It provides a unified view of a model's "
    "development, aiding team analysis. Designed for diverse environments "
    "like scripts or notebooks, it simplifies result logging to files or servers, "
    "enhancing run comparisons for users.",
    score=5,
    grading_context={
        "targets": "MLflow Tracking provides both an API and UI dedicated to the logging "
        "of parameters, code versions, metrics, and artifacts during the ML process. This "
        "centralized repository captures details such as parameters, metrics, artifacts, "
        "data, and environment configurations, giving teams insight into their models’ "
        "evolution over time. Whether working in standalone scripts, notebooks, or other "
        "environments, Tracking facilitates the logging of results either to local files or a "
        "server, making it easier to compare multiple runs across different users."
    },
    justification="The answer gives a correct summary of MLflow tracking. "
    "The answer does not include any innacuracies or significant omissions.",
)

example2 = EvaluationExample(
    input="What is the MLflow Model Registry?",
    output="MLflow's Model Registry is a version control hub for managing "
    "ML model versions. It helps track model stages and facilitates a "
    "smooth transition to production with a centralized model store and UI.",
    score=3,
    grading_context={
        "targets": "A systematic approach to model management, the Model Registry assists "
        "in handling different versions of models, discerning their current state, and "
        "ensuring a smooth transition from development to production. It offers a centralized "
        "model store, APIs, and UI to collaboratively manage an MLflow Model’s full lifecycle, "
        "including model lineage, versioning, stage transitions, and annotations."
    },
    justification="The output inaccurately characterizes the Model Registry as a version control "
    "only system, thereby failing to capture its broader role in the full lifecycle management "
    "of machine learning models. This overlooks key features such as collaboration, annotations, "
    "and comprehensive lifecycle management, leading to a deduction in the correctness score.",
)

example3 = EvaluationExample(
    input="What is automatic logging in MLflow?",
    output="Automatic logging in MLflow is an AI-driven feature for optimizing data storage, "
    "leveraging algorithms to enhance data retrieval and backups within the ML workflow.",
    score=1,
    grading_context={
        "targets": "Automatic logging allows you to log metrics, parameters, and models "
        "without the need for explicit log statements. There are two ways to use autologging: Call "
        "mlflow.autolog() before your training code. This will enable autologging for each supported "
        "library you have installed as soon as you import it. Use library-specific autolog calls for "
        "each library you use in your code."
    },
    justification="The output erroneously represents automatic logging as a data storage optimization "
    "feature, which is entirely incorrect. Automatic logging in MLflow is actually designed to log "
    "metrics, parameters, and models automatically during the machine learning model training process. "
    "This significant misstatement of MLflow's functionality warrants the score of 1.",
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_correctness_metric = answer_correctness(
    model="openai:/gpt-4", examples=[example1, example2]
)

print(answer_correctness_metric)

EvaluationMetric(name=answer_correctness, greater_is_better=True, long_name=answer_correctness, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_correctness based on the input and output.
A definition of answer_correctness and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Answer correctness is evaluated on the accuracy of the provided output based on the provided targets, which is the ground truth. Scor

In [3]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "What is MLflow?",
            "What is Delta Lake?",
            "How to exit vim?",
            "How to exit emacs?",
        ],
        "ground_truth": [
            "MLflow is an open source platform for managing the end-to-end machine learning lifecycle.",
            "Delta Lake is an open-source storage layer that brings ACID transactions to Apache Spark and big data workloads.",
            "To exit vim, press ESC to enter command mode, then type :q and press Enter.",
            "To exit emacs, press Ctrl+x, then Ctrl+c.",
        ],
    }
)

In [4]:
import mlflow

with mlflow.start_run() as run:
    system_prompt = "Concisely answer the following question."
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-3.5-turbo",
        task=openai.ChatCompletion,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
        targets="ground_truth",
        extra_metrics=[
            answer_correctness_metric,
        ],
    )

results.metrics

2023/11/06 14:56:31 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/11/06 14:56:31 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/11/06 14:56:36 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


  0%|          | 0/1 [00:00<?, ?it/s]

2023/11/06 14:56:47 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/11/06 14:56:47 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/11/06 14:56:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/11/06 14:56:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/11/06 14:56:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/11/06 14:56:48 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: answer_correctness


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00042040476910187863,
 'toxicity/v1/variance': 8.061732311598358e-08,
 'toxicity/v1/p90': 0.0007380339549854398,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 10.4,
 'flesch_kincaid_grade_level/v1/variance': 11.745,
 'flesch_kincaid_grade_level/v1/p90': 13.84,
 'ari_grade_level/v1/mean': 11.375,
 'ari_grade_level/v1/variance': 20.481875,
 'ari_grade_level/v1/p90': 16.11,
 'exact_match/v1': 0.0,
 'answer_correctness/v1/mean': 4.5,
 'answer_correctness/v1/variance': 0.25,
 'answer_correctness/v1/p90': 5.0}

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
df = results.tables["eval_results_table"]
df[
    [
        "inputs",
        "outputs",
        "answer_correctness/v1/score",
        "answer_correctness/v1/justification",
    ]
].style

,inputs,outputs,answer_correctness/v1/score,answer_correctness/v1/justification
0,What is MLflow?,"MLflow is an open-source platform used for managing the machine learning lifecycle. It provides tracking of experiments, packaging and reproducibility of models, and deployment of models to different execution environments.",5,"The output accurately describes MLflow as an open-source platform used for managing the machine learning lifecycle, including tracking of experiments, packaging and reproducibility of models, and deployment of models to different execution environments. This aligns closely with the provided targets, demonstrating a high degree of semantic similarity and factual correctness. There are no significant omissions or inaccuracies in the output. Therefore, the answer_correctness score is 5."
1,What is Delta Lake?,"Delta Lake is an open-source storage layer that brings reliability and scalability to data lakes. It provides ACID transactions, schema enforcement, and metadata management capabilities for big data workloads.",4,"The output correctly identifies Delta Lake as an open-source storage layer that brings reliability and scalability to data lakes. It also correctly mentions that it provides ACID transactions, schema enforcement, and metadata management capabilities for big data workloads. However, it omits the specific mention of Apache Spark, which is included in the target information. This omission is minor, but it does prevent the output from fully aligning with the target information. Therefore, the answer is mostly correct, but with a minor omission."
2,How to exit vim?,"To exit vim, you can press the Esc key to enter command mode, and then type :q! to quit without saving or :wq to save changes and quit. Press Enter to execute the command.",4,"The output is mostly correct and aligns with the provided targets. It correctly explains how to exit vim by pressing the Esc key to enter command mode and then typing :q! to quit without saving or :wq to save changes and quit. However, the output includes an additional command (:q!) that is not mentioned in the targets. This is a minor discrepancy, but it does not significantly affect the overall correctness of the output. Therefore, the output receives a score of 4."
3,How to exit emacs?,"To exit Emacs, you can press the Ctrl and x keys together, followed by the Ctrl and c keys together.",5,"The output provided by the model is completely accurate and matches the target information. The model correctly states that to exit Emacs, one needs to press Ctrl+x, then Ctrl+c. There are no inaccuracies or omissions in the model's output."


## Answer Relevance

In [6]:
from mlflow.metrics.genai import answer_relevance

example1 = EvaluationExample(
    input="What is MLflow Tracking?",
    output="MLflow Tracking provides both an API and UI dedicated to the logging "
    "of parameters, code versions, metrics, and artifacts during the ML process. "
    "This centralized repository captures details such as parameters, metrics, "
    "artifacts, data, and environment configurations, giving teams insight into their "
    "models’ evolution over time.",
    score=5,
    justification="The answer directly addresses the input question and  "
    "provides a concise and clear description of MLflow Tracking.",
)

example2 = EvaluationExample(
    input="What is MLflow Model Registry?",
    output="MLflow Model Registry is a component of MLflow that helps in managing "
    "and deploying models in production. It provides versioning and stage transitions. "
    "MLflow also has a model evaluation feature for evaluating ML models.",
    score=3,
    justification="The answer provides a general idea about MLflow Model Registry and "
    "includes correct details about versioning and stage transitions. The mention of model evaluation "
    "is irrelevant to the input question, hence the score of 3.",
)

example3 = EvaluationExample(
    input="What is automatic logging in MLflow?",
    output="Delta Lake is an open-source storage layer that brings ACID transactions to Apache "
    "Spark and big data workloads.",
    score=1,
    justification="The output is completely irrelevant to the input question about "
    "automatic logging in MLflow, hence the score of 1.",
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_relevance_metric = answer_relevance(
    model="openai:/gpt-4", examples=[example1, example2, example3]
)

print(answer_relevance_metric)

EvaluationMetric(name=answer_relevance, greater_is_better=True, long_name=answer_relevance, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_relevance based on the input and output.
A definition of answer_relevance and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Answer relevance measures the appropriateness and applicability of the output with respect to the input. Scores should reflect the extent to 

In [7]:
import mlflow

with mlflow.start_run() as run:
    system_prompt = "Concisely answer the following question."
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-3.5-turbo",
        task=openai.ChatCompletion,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
        targets="ground_truth",
        extra_metrics=[
         answer_relevance_metric,
        ],  # use the answer similarity metric created above
    )

results.metrics

/Users/daniel.liden/miniconda3/envs/mlops/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/daniel.liden/miniconda3/envs/mlops/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/11/06 14:56:59 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/11/06 14:56:59 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/11/06 14:57:13 INFO mlflow.models.evaluation.default_evaluator: Te

  0%|          | 0/1 [00:00<?, ?it/s]

2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/11/06 14:57:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: answer_relevance


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.0003918855873052962,
 'toxicity/v1/variance': 6.071496609846221e-08,
 'toxicity/v1/p90': 0.0006513843894936144,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 8.55,
 'flesch_kincaid_grade_level/v1/variance': 20.232499999999998,
 'flesch_kincaid_grade_level/v1/p90': 13.350000000000001,
 'ari_grade_level/v1/mean': 10.0,
 'ari_grade_level/v1/variance': 39.995000000000005,
 'ari_grade_level/v1/p90': 16.66,
 'exact_match/v1': 0.0,
 'answer_relevance/v1/mean': 5.0,
 'answer_relevance/v1/variance': 0.0,
 'answer_relevance/v1/p90': 5.0}

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
df = results.tables["eval_results_table"]

df[["inputs", "outputs", "answer_relevance/v1/score", "answer_relevance/v1/justification"]].style

,inputs,outputs,answer_relevance/v1/score,answer_relevance/v1/justification
0,What is MLflow?,MLflow is an open-source platform used for managing the complete machine learning lifecycle.,5,"The output directly addresses the input question and provides a concise and clear description of MLflow. It is completely relevant to the question asked, hence the score of 5."
1,What is Delta Lake?,"Delta Lake is an open-source storage layer that provides reliability, scalability, and performance improvements to data lakes. It allows for ACID transactions, schema enforcement, and data versioning, making it easier to build robust and scalable data pipelines for analytics and machine learning.",5,"The output provides a comprehensive and accurate answer to the input question about Delta Lake. It mentions that Delta Lake is an open-source storage layer and goes on to describe its key features and benefits, such as reliability, scalability, performance improvements, ACID transactions, schema enforcement, and data versioning. All parts of the output are meaningful and relevant to the question, hence the score of 5."
2,How to exit vim?,"To exit vim, you can use the command :q or :wq to save and quit, or :q! to force quit without saving.",5,"The output directly answers the input question by providing the commands to exit vim. It also provides additional relevant information about how to save and quit, or force quit without saving, which are all relevant to the question. Therefore, the answer is complete and directly relevant to the question, hence the score of 5."
3,How to exit emacs?,"To exit Emacs, you can use the command C-x C-c.",5,"The output directly answers the input question by providing the exact command to exit Emacs. All parts of the output are meaningful and relevant to the question, hence the score of 5."


## Faithfulness
The [*faithfulness*](https://mlflow.org/docs/latest/python_api/mlflow.metrics.html#mlflow.metrics.genai.faithfulness) metric assesses "how factually consistent the output is to the context." So for this metric, we require input, output, and context.

In [13]:
from mlflow.metrics.genai import faithfulness, EvaluationExample

# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="What is MLflow Tracking?",
        output="MLflow Tracking offers an API and UI for centralized logging of machine "
        "learning parameters, code versions, metrics, and artifacts, supporting a variety "
        "of environments and enabling easier comparison of multiple runs across users.",
        score=5,
        justification="The output provides a clear answer including only details from the provided context",
        grading_context={
            "context": "MLflow Tracking provides both an API and UI dedicated to the logging "
            "of parameters, code versions, metrics, and artifacts during the ML process. This "
            "centralized repository captures details such as parameters, metrics, artifacts, "
            "data, and environment configurations, giving teams insight into their models’ "
            "evolution over time. Whether working in standalone scripts, notebooks, or other "
            "environments, Tracking facilitates the logging of results either to local files or a "
            "server, making it easier to compare multiple runs across different users."
        },
    ),
    EvaluationExample(
        input="What is MLflow Model Registry?",
        output="The Model Registry centralizes MLflow Models in a repository, offering APIs, a "
        "user interface, and features like versioning, state tracking, and annotations, as "
        "well as the ability to archive, delete, and search models for a seamless transition "
        "from development to production and ongoing management.",
        score=3,
        justification="The output receives a score of 3 rather than a lower score like 1 "
        "because it does accurately reflect most of the core details mentioned in the "
        "original context such as centralized storage, APIs, UI, versioning, state tracking, "
        "and annotations. However, it introduces extraneous details like the ability to "
        "'archive, delete, and search models,' which are not derived from the context. "
        "Additionally, it substitutes 'repository' for 'store' and introduces 'ongoing "
        "management,' slight deviations from the original description. These inaccuracies "
        "and additions are not fundamentally wrong, but they are not faithful to the original "
        "context, warranting a deduction in score.",
        grading_context={
            "context": "A systematic approach to model management, the Model Registry assists "
            "in handling different versions of models, discerning their current state, and "
            "ensuring a smooth transition from development to production. It offers a centralized "
            "model store, APIs, and UI to collaboratively manage an MLflow Model’s full lifecycle, "
            "including model lineage, versioning, stage transitions, and annotations."
        },
    ),
    EvaluationExample(
        input="What is automatic logging in MLflow?",
        output="Automatic logging in MLflow is a machine learning technique used to automate the "
        "process of cutting down trees for lumber.",
        score=1,
        justification="The output is entirely incorrect and fails to utilize the given context. "
        "The context is about MLflow's automatic logging for metrics, parameters, and models in "
        "machine learning projects. Instead, the output discusses using machine learning to automate "
        "the process of cutting down trees, which is completely unrelated to the actual context provided.",
        grading_context={
            "context": "Automatic logging allows you to log metrics, parameters, and models without the "
            "need for explicit log statements. There are two ways to use autologging: Call mlflow.autolog() "
            "before your training code. This will enable autologging for each supported library you have "
            "installed as soon as you import it. Use library-specific autolog calls for each library you use "
            "in your code."
        },
    ),
]

faithfulness_metric = faithfulness(
    model="openai:/gpt-4", examples=faithfulness_examples[1:2]
)
print(faithfulness_metric)

EvaluationMetric(name=faithfulness, greater_is_better=True, long_name=faithfulness, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called faithfulness based on the input and output.
A definition of faithfulness and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Faithfulness is only evaluated with the provided output and provided context, please ignore the provided input entirely when scoring faithfulness. Faithfulne

In [14]:
eval_df = pd.DataFrame(
    {
        "question": [
            "What is MLflow?",
            "What is Delta Lake?",
            "How to exit vim?",
            "How to exit emacs?",
        ],
        "context": [
            "MLflow, at its core, provides a suite of tools aimed at simplifying the ML "
            "workflow. It is tailored to assist ML practitioners throughout the various "
            "stages of ML development and deployment. Despite its expansive offerings, "
            "MLflow’s functionalities are rooted in several foundational components:"
            "Tracking: MLflow Tracking provides both an API and UI dedicated to the "
            "logging of parameters, code versions, metrics, and artifacts during the ML "
            "process. This centralized repository captures details such as parameters, "
            "metrics, artifacts, data, and environment configurations, giving teams "
            "insight into their models’ evolution over time. Whether working in standalone "
            "scripts, notebooks, or other environments, Tracking facilitates the logging "
            "of results either to local files or a server, making it easier to compare "
            "multiple runs across different users."
            "Model Registry: A systematic approach to model management, the Model Registry "
            "assists in handling different versions of models, discerning their current "
            "state, and ensuring a smooth transition from development to production. It "
            "offers a centralized model store, APIs, and UI to collaboratively manage an "
            "MLflow Model’s full lifecycle, including model lineage, versioning, stage "
            "transitions, and annotations."
            "AI Gateway: This server, equipped with a set of standardized APIs, streamlines "
            "access to both SaaS and OSS LLM models. It serves as a unified interface, "
            "bolstering security through authenticated access, and offers a common set of "
            "APIs for prominent LLMs."
            "Evaluate: Designed for in-depth model analysis, this set of tools facilitates "
            "objective model comparison, be it traditional ML algorithms or cutting-edge "
            "LLMs."
            "Prompt Engineering UI: A dedicated environment for prompt engineering, this "
            "UI-centric component provides a space for prompt experimentation, refinement, "
            "evaluation, testing, and deployment."
            "Recipes: Serving as a guide for structuring ML projects, Recipes, while "
            "offering recommendations, are focused on ensuring functional end results "
            "optimized for real-world deployment scenarios."
            "Projects: MLflow Projects standardize the packaging of ML code, workflows, "
            "and artifacts, akin to an executable. Each project, be it a directory with "
            "code or a Git repository, employs a descriptor or convention to define its "
            "dependencies and execution method.",
            "Delta Lake is an open source project that enables building a Lakehouse "
            "architecture on top of data lakes. Delta Lake provides ACID transactions, "
            "scalable metadata handling, and unifies streaming and batch data processing "
            "on top of existing data lakes, such as S3, ADLS, GCS, and HDFS."
            "Specifically, Delta Lake offers:"
            "ACID transactions on Spark: Serializable isolation levels ensure that readers "
            "never see inconsistent data."
            "Scalable metadata handling: Leverages Spark distributed processing power to "
            "handle all the metadata for petabyte-scale tables with billions of files at ease."
            "Streaming and batch unification: A table in Delta Lake is a batch table as well "
            "as a streaming source and sink. Streaming data ingest, batch historic backfill, "
            "interactive queries all just work out of the box."
            "Schema enforcement: Automatically handles schema variations to prevent insertion "
            "of bad records during ingestion."
            "Time travel: Data versioning enables rollbacks, full historical audit trails, "
            "and reproducible machine learning experiments."
            "Upserts and deletes: Supports merge, update and delete operations to enable "
            "complex use cases like change-data-capture, slowly-changing-dimension (SCD) "
            "operations, streaming upserts, and so on.",
            "After saving your changes, you can quit Vim with :q. Or the saving and "
            "quitting can be combined into one operation with :wq or :x."
            "If you want to discard any changes, enter :q! to quit Vim without saving.",
            "C-x C-c\n\n    Kill Emacs (save-buffers-kill-terminal)."
            "C-z\n\n    On a text terminal, suspend Emacs; on a graphical display, "
            "iconify (or “minimize”) the selected frame (suspend-frame)."
            "Killing Emacs means terminating the Emacs program. To do this, type "
            "C-x C-c (save-buffers-kill-terminal). A two-character key sequence is "
            "used to make it harder to type by accident. If there are any modified "
            "file-visiting buffers when you type C-x C-c, Emacs first offers to save "
            "these buffers. If you do not save them all, it asks for confirmation "
            "again, since the unsaved changes will be lost. Emacs also asks for "
            "confirmation if any subprocesses are still running, since killing "
            "Emacs will also kill the subprocesses (see Running Shell Commands from Emacs)."
            "C-x C-c behaves specially if you are using Emacs as a server. If you "
            "type it from a client frame, it closes the client connection. See Using "
            "Emacs as a Server.",
        ],
    }
)


In [15]:
with mlflow.start_run() as run:
    system_prompt = "Concisely answer the following question using only the information provided in the context."
    basic_context_model = mlflow.openai.log_model(
        model="gpt-3.5-turbo",
        task=openai.ChatCompletion,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Context:\n{context}\n\nQuestion:\n{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_context_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        predictions="result",
        evaluator_config={
            "col_mapping": {
                "inputs": "question",
                "context": "context"
            }
        },
        extra_metrics=[faithfulness_metric],  # use the faithfulness metric
    )

results.metrics

/Users/daniel.liden/miniconda3/envs/mlops/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/daniel.liden/miniconda3/envs/mlops/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/11/06 14:59:46 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/11/06 14:59:46 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/11/06 15:00:13 INFO mlflow.models.evaluation.default_evaluator: Te

  0%|          | 0/1 [00:00<?, ?it/s]

2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/11/06 15:00:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: faithfulness


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.0005084216718387324,
 'toxicity/v1/variance': 1.2682792350723286e-07,
 'toxicity/v1/p90': 0.0008852531376760453,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 13.525,
 'flesch_kincaid_grade_level/v1/variance': 147.90187500000002,
 'flesch_kincaid_grade_level/v1/p90': 26.720000000000006,
 'ari_grade_level/v1/mean': 17.724999999999998,
 'ari_grade_level/v1/variance': 207.89687500000002,
 'ari_grade_level/v1/p90': 33.290000000000006,
 'faithfulness/v1/mean': 5.0,
 'faithfulness/v1/variance': 0.0,
 'faithfulness/v1/p90': 5.0}

In [17]:
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
df = results.tables["eval_results_table"]

# just show question outputs faithfulness/v1/score	faithfulness/v1/justification columns
df[
    [
        "question",
        "outputs",
        "faithfulness/v1/score",
        "faithfulness/v1/justification",
    ]
].rename(
    columns={
        "faithfulness/v1/score": "score",
        "faithfulness/v1/justification": "justification",
    }
).style

,question,outputs,score,justification
0,What is MLflow?,"MLflow is a suite of tools that simplifies the machine learning workflow by providing functionalities such as tracking and logging of parameters, metrics, and artifacts during the ML process, managing different versions of models, streamlining access to both SaaS and OSS LLM models, facilitating model analysis and comparison, providing a UI for prompt engineering, offering recommendations for structuring ML projects, and standardizing the packaging of ML code, workflows, and artifacts.",5,"The output accurately reflects all the core details mentioned in the original context. It mentions tracking and logging of parameters, metrics, and artifacts during the ML process, managing different versions of models, streamlining access to both SaaS and OSS LLM models, facilitating model analysis and comparison, providing a UI for prompt engineering, offering recommendations for structuring ML projects, and standardizing the packaging of ML code, workflows, and artifacts. All these points are directly supported by the provided context, demonstrating high faithfulness to the provided context. Therefore, the output gets a score of 5."
1,What is Delta Lake?,"Delta Lake is an open source project that allows for the creation of a Lakehouse architecture on top of data lakes. It provides features such as ACID transactions, scalable metadata handling, and unifies streaming and batch data processing.",5,"The output accurately reflects all the core details mentioned in the original context. It correctly identifies Delta Lake as an open source project that enables the creation of a Lakehouse architecture on top of data lakes. It also correctly mentions the features of Delta Lake such as ACID transactions, scalable metadata handling, and the unification of streaming and batch data processing. There are no extraneous details or inaccuracies in the output, and it does not introduce any information that is not derived from the context. Therefore, the output demonstrates high faithfulness to the provided context."
2,How to exit vim?,"After saving your changes, you can quit Vim with :q.",5,"The output is completely faithful to the context provided. The output states ""After saving your changes, you can quit Vim with :q."" which is exactly what the context says. There are no additional claims or contradictions in the output that are not supported by the context. Therefore, the faithfulness score is 5."
3,How to exit emacs?,"To exit Emacs, you can use the key sequence C-x C-c (save-buffers-kill-terminal).",5,"The output is completely faithful to the context provided. The output states ""To exit Emacs, you can use the key sequence C-x C-c (save-buffers-kill-terminal)."" This is directly supported by the context, which states ""To do this, type C-x C-c (save-buffers-kill-terminal)."" Therefore, all of the claims in the output can be directly inferred from the context, warranting a score of 5."
